# Exploring Jane Addams Letters 1901-1927

### MATH-390 | 01 | ADV TOPICS: NETWORK SCIENCE | Professor Beecher

### Tina Nosrati
### last update: 12/01/2024


In [4]:
#packages
import pandas as pd
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.colors as mcolors
import community as community_louvain
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize

In [23]:
nodes=pd.read_csv('cleaned_data.csv')
df_all=pd.read_csv("similarity_df_complete.csv")
df_1901_1909=pd.read_csv("df_1901_1909.csv")
df_1910_1919=pd.read_csv("df_1910_1919.csv")
df_1920_1927=pd.read_csv("df_1920_1927.csv")

In [9]:
nodes

,Unnamed: 0,Identifier,Title,Subject,Date,Text,locations,tags
0,0,43341,"Jane Addams to Zonja Wallin Lawrence, May 7, 1929","Hull-House, support for;Addams, Jane, gratitude",1927-05-07,hullhous 800 south halst street chicago dear m...,"41.8716164,-87.6474573",Hull-House;Thanks
1,19,42487,"Jane Addams to Dorothy Detzer, ca. October 2, ...",Women's International League for Peace and Fre...,1926-10-02,woman intern leagu peac freedom intern offic g...,"41.8781136,-87.6297982",Finances;Peace
2,20,42486,"Jane Addams to Hannah Clothier Hull, ca. Octob...","Addams, Jane, requests from;Addams, Jane, and ...",1926-10-02,woman intern leagu peac freedom intern offic g...,"41.8781136,-87.6297982",Peace;Requests
3,21,42485,"Jane Addams to Myra Harriet Reynolds Linn, Sep...","Addams, Jane, and family",1926-09-27,12 west walton place sept 271926 dear myra sen...,"41.9001602,-87.6290308",Family
4,24,42466,"Jane Addams to Madeleine Zabriskie Doty, Augus...","Addams, Jane, requests from;Women's Internatio...",1926-08-31,dear madelein doti note given cours know noth ...,"46.2043907,6.1431577",Peace;Requests
...,...,...,...,...,...,...,...,...
1311,3888,103,"Jane Addams to Richard Theodore Ely, February ...","Addams, Jane, requests from;Hull-House, visits to",1901-02-26,hullhous 335 south halst street chicago dear m...,"41.8716782,-87.6474517",Hull-House Visits;Requests
1312,3893,76,"Jane Addams to Florence Kelley, February 10, 1901","Addams, Jane, relationship with Florence Kelle...",1901-02-10,hullhous 335 south halst street chicago dear s...,"41.8716782,-87.6474517",Friends
1313,3897,63,"Jane Addams to Sarah Alice Addams Haldeman, Ja...","Addams, Jane, and family",1901-01-30,hullhous 335 south halst street chicago dear a...,"41.8716782,-87.6474517",Family
1314,3898,59,"Jane Addams to Anita McCormick Blaine, January...","Addams, Jane, writing",1901-01-17,hullhous 335 south halst street chicago dear m...,"41.8716782,-87.6474517",Lectures


In [19]:
def topic_modeling(df, num_topics=5, num_words=5, letter_words=None):
    # Ensure 'Text' column exists
    if 'Text' not in df.columns:
        raise ValueError("The DataFrame must contain a 'Text' column.")
    
    # Preprocess text: Remove NaN and empty values
    text_data = df['Text'].dropna().tolist()
    
    # Default letter-related words to remove
    letter_words = letter_words or [
        'dear', 'sincerely', 'regards', 'yours', 'truly', 
        'letter', 'address', 'correspondence', 'missive', 
        'page', 'send', 'numbers','chicago', 'mr', 'hullhous', 'street', 'addam', 'dear', 'miss',
        'south', 'halst', '800', 'sign'
    ]
    
    # Vectorize text with custom stop words
    vectorizer = CountVectorizer(stop_words=letter_words, max_df=0.95, min_df=2)
    dtm = vectorizer.fit_transform(text_data)
    
    # Fit LDA model
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(dtm)
    
    # Extract topics
    topics = []
    feature_names = vectorizer.get_feature_names_out()
    for idx, topic in enumerate(lda.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]
        topics.append(f"Topic {idx + 1}: " + ", ".join(top_words))
    
    return topics


In [21]:
topics = topic_modeling(nodes)
print(topics)

['Topic 1: jane, hastili, one, alway, may', 'Topic 2: sincer, may, jane, thank, faith', 'Topic 3: jane, alway, love, come, much', 'Topic 4: peac, woman, meet, parti, committe', 'Topic 5: would, intern, offic, woman, peac']


Personal Reflections or Daily Musings

Gratitude and Faith

Love and Relationships

Peace and Social Gatherings

Women’s Peace Efforts

In [24]:
topics = topic_modeling(df_1901_1909)
print(topics)

['Topic 1: jane, 335, much, one, come', 'Topic 2: go, come, alway, love, summer', 'Topic 3: aug, 335, jane, year, faith', 'Topic 4: 335, jane, may, come, march', 'Topic 5: love, alic, marcet, come, alway']


Jane and Personal Reflections

Seasonal Experiences and Affection

Faith and Yearly Events

Dates and Jane's Activities

Love and Relationships

In [25]:
topics = topic_modeling(df_1910_1919)
print(topics)

['Topic 1: jane, much, sincer, hope, new', 'Topic 2: peac, woman, offic, secretari, parti', 'Topic 3: jane, would, think, new, much', 'Topic 4: jane, alway, love, hope, come', 'Topic 5: meet, peac, woman, new, committe']


Jane's Hopes and Sincerity

Women's Peace and Organizational Work

Jane's Thoughts and Reflections

Love, Hope, and Personal Connections

Meetings and Peace Committees

In [26]:
topics = topic_modeling(df_1920_1927)
print(topics)

['Topic 1: jane, faith, new, year, thank', 'Topic 2: intern, would, jane, woman, one', 'Topic 3: jane, love, alway, aunt, littl', 'Topic 4: intern, peac, woman, may, leagu', 'Topic 5: intern, geneva, woman, jane, sent']


Gratitude, Faith, and New Beginnings

Women's Role in International Affairs

Love, Family, and Personal Bonds

International Peace and Women's League

Geneva and International Communication